<b>For use with:</b> <font color="red" ><b>Alpaca Kernel</b></font>
<br>
Accessibility of this notebook  

    
>- Adjust the width of the text with the function provided in the cell below. 
>- Toggle the auto-numbering of the sections in the outline toolbox (sidebar or topbar).
>- Toggle the code line numbers in the dropdown menu of the "view" button in the topbar. 
>- Collapse/expand a cell by clicking the blue bar on the left side of the cell.



In [ ]:
from IPython.core.display import HTML
def set_width(width):
    display(HTML(f"""<style>  
            .container {{ width:{width}% !important; 
                            min-width:800px !important; margin: 0 auto}} 
            .jp-Cell {{ width:{width}% !important; 
                            min-width:800px !important; margin: 0 auto}} </style>"""))
# Set container width to X%v of the fullscreen 
set_width(50)

**Voltammetry - Final Project**

<font size="3"><b> Goals - Overview:</b></font>

- Week 17: Understand the design limitations and build a simple model of a potentiostat<br>
- **Week 18: Build the potentiostat and write code for voltammetric measurements <br>**
- Week 19: Run voltammetric measurements with different techniques and samples <br>

<font size="3"><b> Experiments - Week 18:</b></font>

- 18A: Cyclic Voltammetry - Part 1 - Data Acquisition
- 18B: Cyclic Voltammetry - Part 2 - Data Evaluation
- 18o: Optional Voltammetric Techniques

# 17A - Potentiostat: Design and Limitations - Introduction

## Goals
   
   >- Get an overview of the proposed implementations of a Potentiostat on Alpaca with Pico Pi
   >- Understand the challenges, limitations, pros and cons in suggested designs
   >- Choose a design for your implementation
   >- Learn how to control the potentiostat

## Structure

   >- Background: 90 min
   >- Anticipate: 5 + 60 + 30 min
   >- Simulate:   10 + 20 + 30 min
   >- Conclude and Choose: 15 min

## Check-off rules
   
   >- If you are still unsure which design to choose at the beginning of the *Classroom session*, feel free to reach out to the TAs and discuss which option would be the most suitable for you and your partner.
   >- Reach out to the TAs for checking off this notebook after completing Background + Anticipate + Simulate and discussing your results, preference and decision with your partner
   >   - Prepare your answers (and if you still have some questions) from the Anticipate section
   >   - Present the simulated Voltammomogram
   >   - Discuss your answers from the Conclude and Choose section

## Optional assignments

1. 🏆 = Entirely optional. Interesting and useful. 
2. 🏆🏆 = Optional. Recommended.
3. 🏆🏆🏆 = Recommended. It might be challenging, but not necessarily.
4. 🏆🏆🏆🐐 = Challenging. Surely, it's worth it!

# Background


## The Measurement Cell

There are many approaches to constructing measurement cells. Historically, these were made from glassware, but modern techniques increasingly favor planar designs. Planar construction allows for electrodes to be screen-printed, resulting in cost-effective and disposable sensors.

### Sensor

Below are examples of such planar sensors, which will also be used in our experiments:

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/sensors.JPG" width=500><br>
<em>Sensors</em>
</div><br>

These sensors are printed on a foil substrate, except for the Metrohm sensor, which is printed on a ceramic base. The brown strip on the right acts as a support strip, providing mechanical stability for the foil sensors.

All sensors share a standard connection method, as shown below:

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/sensor_connection.jpg" width=500><br>
<em>Sensor Connections</em>
</div><br>

At the top is the Voltammetry Adapter, designed for use with a breadboard. Below it is the sensor. Since foil sensors are only 0.2 mm thick, the brown strip serves as a filler, ensuring mechanical stability when the sensor is inserted into the adapter.

The adapter includes a GND connection linked to a copper plane on the circuit board. When grounded, this acts as a shield to minimize interference from external sources, such as the 50 Hz noise from power lines.

In the adapter, pairs of pins are connected in parallel, making them compatible with standard breadboard contact arrangements.

Below is an example of how the adapter connects to the ALPACA system:

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/sensor_ALPACA.JPG" width=500><br>
<em>Sensor Connected to the ALPACA</em>
</div>

Finally, detailed photographs of the Voltammetry Adapter board are provided below:

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/voltammetry_adapter_board.JPG" width=500><br>
<em>Voltammetry Adapter Board</em>
</div>

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/voltammetry_adapter_board_backside.JPG" width=500><br>
<em>Voltammetry Adapter Board - Back Side (Yellow = Support)</em>
</div>

### Cell Models

In the development of measurement systems, extensive testing is often required. When testing procedures focus on the electronic components, it is often practical to simplify or bypass the processes occurring in the electrochemical measurement cell. This can be achieved by replacing the chemical cell with a simple, predictable circuit.

Below are two example models for this purpose:

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/CellModels.png" width=1200><br>
    <em>Cell Models</em>
</div><br>

#### Measurement Cell Model 1 - Linear 

This basic model uses two resistors, the values of which can be selected based on the specific testing requirements. For our current setup, resistor values in the range of 1 kΩ to 10 kΩ are appropriate. This model provides a straightforward and predictable behavior, ideal for initial testing of electronic components.

#### Measurement Cell Model 2 - Non-linear

This model introduces a more complex, non-linear behaviour by incorporating a Zener diode. In practice, this means that the signal from the potentiostat will be distinctly different for a negative and a positive cell potential, unlike in Model 1. This response mimics the measurement of a netural *Buffer solution* and can be very useful and significantly faster for testing of your complete setup. For example, for testing your *Cyclic Voltammetry* code.

##### Replacements for the Zener diode

>When the goal is to achieve an asymmetric response of the Measurement Cell Model, Zener diode can be replaced with various components, for example:
>
>1. a Si-diode ($V_{th\,Diode} = 0.6 V$) and a LED ($V_{th\,LED} = 2 V$) <br>connected in parallel - with opposing polarity.
>   
>2. a Si-diode ($V_{th\,1xDiode} = 0.6 V$) connected in parallel to two Si-diodes in series <br> ($V_{th\,2xDiode} = 1.2 V$), but in reverse polarity with respect to the single diode. <br> This combination produces a similar behaviour as LED, but across a smaller range of simulated cell potentials, which might be more practical in some designs or applications.

## Potentiostat - Principle of Operation

In the three-electrode potentiostat, two electrodes are used to conduct the current, while the third electrode is dedicated solely to measuring potential. Since the potential-measuring electrode carries no current, the voltage drop across its resistance in the liquid is zero volts, as per Ohm’s law: $ U = I \cdot R = 0 \cdot R = 0 \, \text{V}. $

The general arrangement of this setup is shown below:

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/Convention.png"><br>
    <em>Potentiostat</em>
</div><br>

In this system:
- The **Counter Electrode (CE)** and the **Working Electrode (WE)** carry the current.
- The current is measured using an Ampere-meter (Am).
- The potential $U_{cell}$ of the **Working Electrode (WE)** is measured relative to the **Reference Electrode (Ref)** using a voltmeter (Vm).
- A variable voltage source $U_{\text{var}}$ powers the entire setup.

The potential of the working electrode ($U_{cell}$) is the parameter controlled in this setup. The voltage source $U_{\text{var}}$ is adjusted to achieve the desired $U_{cell}$. Typically, $U_{cell}$ is not held constant but is swept across a range of voltages defined by the sample under study. 

> Historically, potentiostats required manual operation, with users adjusting $U_{\text{var}}$ while monitoring the voltmeter and recording current measurements. Modern potentiostats, however, are computer-controlled, with a feedback loop that automatically fine-tunes $U_{\text{var}}$ to maintain the desired $U_{cell}$.

### Computer-Controlled Potentiostat

The operation of a computer-controlled potentiostat is illustrated in the diagram below:

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/CompPotentiostat.png"><br>
    <em>Computer-Controlled System</em>
</div><br>

In this setup:
- A voltage source $U_{\text{sweep}}$ can generate negative and positive voltages and sets the desired value for $U_{cell}$.
- A feedback loop, implemented using an operational amplifier (OPAMP1), adjusts the voltage of the CE. This ensures that the measured $U_{\text{cell}}$ reflects $U_{\text{sweep}}$, i.e., $U_{cell} = \text{GND} - U_{\text{sweep}}$ and so:
$$
U_{\text{cell}} = - U_{\text{sweep}}
$$
In the schematic, the Ampere-meter (Am) is idealized to have no voltage drop. However, in real-world applications, a small voltage drop across the Ampere-meter can result in a slight discrepancy between $U_a$ and $U_{\text{sweep}}$.

> The voltage sweep range is chosen based by expected the properties of the sample being analyzed and the composition of the surrounding solution. It often spans from the negative to positive voltages.
>
> **During the sweep, the current $I_{cell}$ can take on positive or negative values, regardless of the value of $U_{cell}$, because it depends on the chemical reaction taking place in the cell and the conditions of the measurement.**
>
> This wide range of possibilities causes a technical challenge for controlling and measuring such a system, and it is the case with Pico Pi and Alpaca.

#### Potentiostat Controlled with Positive Voltages Sources 

The configuration explained in the previous section can be extended by an additional voltage source $U_{\text{offset}}$.

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/CompPotentiostatOffset.png"><br>
    <em>Potentiostat Controlled with two Voltages Sources</em>
</div><br>

Thanks to this, both negative and positive $U_{\text{cell}}$ can be set using voltage sources that output only positive voltages: 
$$
U_{\text{cell}} = U_{\text{offset}} - U_{\text{sweep}}
$$




### Current measurement


Measuring current in a circuit can be accomplished in several ways. One common method involves placing a resistor in series with the circuit and measuring the voltage drop across it. While simple and effective, this technique introduces an additional voltage drop into the circuit, which may lead to errors. Whether this is acceptable depends on the circuit's requirements and the magnitude of the introduced error.

<details>
<summary><font size=3>🏆</font> <b>Optional Challenge</b></summary>

> Implement this technique in your potentiostat and compare it with the suggested designs. Make sure to report your observations.

</details> 
<br>

Preferably, an operational amplifier (OPAMP) can be used as a current-to-voltage converter. This method allows for current measurement with negligible voltage drop. However, the measurement range is limited by the current-handling capacity of the OPAMP.

In voltammetry, where currents typically range from microamperes to milliamperes, OPAMP-based current measurement is highly effective. Below is an example of such a circuit:

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/CompPotConverter.png"><br>
    <em>Current Measurement</em>
</div><br>

In this circuit:
1.  There is only one voltage source to control the potentiostat, which means that it must be able to generate negative voltages, if the positive $U_{\text{cell}}$ is desired, and vice versa. <br>
    <details>
    <summary> <font size=4>💡</font> <b>Hint</b></summary>
    
    > Alpaca's `DAC Assistant` can be used in this case.
    
    </details> 
    <br>

3. The voltage $U_a$ represents the current ($I_{\text{cell}}$).
    -  The cell current ($I_{\text{cell}}$) is converted into a voltage by the OPAMP2 acting as a current-to-voltage converter in combination with the feedback resistor $R_f$.
    -  The value of $R_f$ should be carefully chosen to ensure the current of interest utilizes the full range of the analog-to-digital converter (ADC).
    -   The conversion is described by the equation: $U_{a} = - ( - I_{\text{cell}} ) \cdot R_f$, which simplifies to:
        $$ 
        U_{a} = I_{cell} \cdot R_f
        $$
    -   A positive current results in a positive voltage, and vice versa. <br>
        <details>
        <summary> <font size=4>💡</font> <b>Hint</b></summary>
        
        >If this configuration is implemented on Alpaca, the signal must be measured via the non-inverting and the inverting amplifier using two separate ADCs simulataneously, <br>i.e. `AMP0:SIGNAL+ to ADC0` and `AMP1:SIGNAL- to ADC1`, and then added together.
        
        </details> 
        <br>    

####  Current-to-Voltage Converter with Offset

Additionally, the output range of the OPAMP-based current-to-voltage converter can be adjusted with an offset voltage source $U_{\text{offset}}$ at the non-inverting input of the OPAMP2. 

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/CompPotConverterOffset.png"><br>
    <em>Current Measurement with Offset</em>
</div><br>

In this circuit:
1. The conversion is described by the equation: 
    $$ 
    U_{a} = I_{cell} \cdot R_f + U_{offset}
    $$
2. Positive currents result in positive voltages, but with an offset.
   
4. Additionally, a pre-defined range of negative currents also results in a positive voltages.
   <details>
    <summary> <font size=4>💡</font> <b>Hint</b></summary>
       
   > Note that:
   > - $U_{\text{offset}}$ affects the value of $U_{\text{cell}}$
   > - Setting $U_{\text{offset}} = 0V$ reduces this system to the previous one.
   </details> 

#### Handling Noise 

##### Filtering with a *Tamed* Integrator
One of the easiest to implement and a very effective method to increase the signal-to-noise ratio and to improve the quality of your measurements is to use filters. A simple addition of a capacitor parallel to the feedback line of the current-to-voltage converter creates a low-pass filter between the Measurement Cell and the Voltmeter.

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/CellConverterFIlter.png"><br>
    <em>Current-to-Voltage Converter with a Low-pass Filter</em>
</div><br>

<details>
<summary><font size=4>💡</font> <b>Hint</b></summary>

> Keep in mind that setting the cut-off frequency of the filter too low might obscure some essential information in your results, so use your intuition first to set a cut off frequency for a given set of parameters, and test your setup before running the actual measurement.
</details>

<details>
<summary><font size=3>🏆🏆🏆</font> <b>Optional</b></summary>

> Think at which other points in the circuit of your setup filtering could be applied. It might require some creativity and a closer look at the Alpaca's main circuit diagram. Do let us know your about your ideas before trying! But if you give it a thought, make sure to include that in your report.
</details>

##### Averaging

For the extra bit of noise below the cut-off frequency, averaging over several measurements in a very short time can dramatically improve your results. However, pay attention to the often neglected issue that each measurement takes time, and if it's not accounted for properly in your procedure, your measurement results might deviate from the expectation.

## Potentiostat - Design

The considerations of the previous section can be summarised in three functions of a potentiostat that will guide the design choices:
1. Controlling $U_{\text{cell}}$
2. Controlling $I_{\text{cell}}$-to-$U_{\text{a}}$ conversion
3. Measuring $U_{\text{a}}$

Based on the functionality and the limitations of Alpaca and Pico Pi, we outlined two design options for a Potentiostat. Their features, as well as, some key advantages and disadvantages are presented in this section.

### Basic Design (Design B) 

This design is recommended when ease of use is prioritised.

#### Layout
<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/DesignB+ADC.png"><br>
    <em>Potentiostat - Basic Design</em>
</div><br>

In this design, current measurement is realised via the standard current-to-voltage converter without the offset (`Uoffset = GND`). This approach has the following implications for the implementation on Alpaca:
1. In order to control `Ucell` across a range of positive and negative values, `DAC Assistant` has to be used.
    <details>
    <summary><font size=4>💡</font> <b>Hint</b></summary>

    > Think carefully what is the output of `DAC Assistant: OUT`, when `DACA` is not active, so `DACA=0V`. Check the section *Disconnecting the measurement cell* below for more information.
    </details>
2. `Ua` can be both positive and negative, so to obtain reliable and precise results, this signal has to be measured with two channels simultaneously;
   - one configured to measure the positive part of the signal `AMP0:SIGNAL+ to ADC0`
   - and the other, to measure the negative part of the signal `AMP1:SIGNAL- to ADC1`.<br>
   In this configuration, `AMP0` acts as a non-inverting amplifier and `AMP1` as an inverting amplifier. Further details on the correct settings will be presented later in this assignment. You can also find further instructions in the Alpaca Manual.
3. 🏆🏆🏆 **Optional:** The original `DACA` values before they're converted to `DAC Assistant:OUT` can be measured independently via `Ain2` for reference.

#### Advantages

1. Intuitive control of `Ucell` and evaluation of `Icell`
2. Wide range of `Ucell`, from -10V to ca. +5V
3. Wide range of `Icell`, thanks to the dual channel measurement

#### Disadvantages

1. Lower accuracy of and lower resolution of `Ucell` control
2. Lower accuracy for measurements of low voltages `-100mV<Ua<100mV` because of the dual-channel measurement
3. Higher total noise
4. `Relay` must be used to disconnect the Measurement Cell at idle operation

<details>
<summary><font size=4>🔌<b>Disconnecting the Measurement Cell</b></font></summary>
<br>
The electrochemical reaction within the measurement cell continues as long as a current can flow. To prevent undesired reactions during idle periods, it's crucial to make sure that the signals applied to the cell do not generate a substantial current.

This can be achieved by setting the voltage to the cell to zero, by controlling the voltage in the potentiometric regulation loop. This comes with its own challenges in this design, which you will discover during the test procedures.

Alternatively, the relay on the ALPACA board can be used to electrically disconnect the measurement cell from the control setup. The relay is located on the ALPACA board, close to the construction site. It can be controlled using the pin labelled `Relay` on `J8`. The relay will react (energize the electro-magnet) when a signal 3.3V or 5V is applied to the pin “relay”.

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/diconnect_via_relay.JPG" width=400><br>
    <em>ALPACA's Relay</em>
</div><br>

>In its **default state**, the electromagnet is not energized:
>- the relay connects the `COMMON` terminal to the `NORMAL CLOSED`. <br>
>
>In its **energized state**, the electromagnet switches this connection:
>- and the relay links the `COMMON` terminal with `NORMAL OPEN`, effectively connecting the measurement cell.
>
>
>
> Use the `GPIO14` pin of `J1` on Cria to control the action of the relay. <br>
> You can instantiate and manipulate the Python object for the relay with:
> ```python
> from machine import Pin
> relay = Pin(14, Pin.OUT)
> relay.value(0) ## OFF
> relay.value(1) ## ON
> ``` 

Have a look into the Alpaca Manual (Download from Brightspace) for more info on the use of the relay and **a complete Alpaca circuit layout on the last page of the manual.**


</details>


### Design A (Advanced)

This design is recommended for high quality measurements.

#### Layout

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/DesignA+ADC.png"><br>
    <em>Potentiostat - Advanced Design</em>
</div><br>

In this design, current measurement is realised via the current-to-voltage converter with offset (`Uoffset = DACB`). This approach has the following implications for the implementation on Alpaca:
1. In order to control `Ucell` across a range of positive and negative values, `DACA` and `DACB` are used directly in combination. They can be configured such that `Ucell` takes positive or negative values.
2. The offset `Uoffset` can be set such that `Ua` takes only positive values for a range of currents, negative and positive, and thus, `Ua` can be measured with a single analog-to-digital converter, for example: `ADC0` via `AMP0:SIGNAL+`.
3. 🏆🏆🏆 **Optional:** `DACA` and `DACB` can be measure independently via `Ain1` or `AMP1:SIGNAL+` and `Ain2` for reference.

#### Advantages

1. Higher accuracy and a higher resolution of `Ucell` control.
2. Precise measurement with a single ADC across a range of negative and positive cell currents.
3. Lower total noise.
4. 🏆🏆🏆🐐 **Optional:** Range of `Ua` can be adjusted programatically.
5. 🏆🏆🏆🐐 **Optional:** `Relay` is available and can be used for switching `Rf` on demand.

#### Disadvantages

1. Control of `Ucell` and evaluation of `Icell` are slightly more complicated
2. Lower range of `Ucell`
3. Lower range of `Icell`
4. Depending on the control procedure the range of `Icell` measured via `Ua` might be correlated with the value of `Ucell`  


# Anticipate

## A1: Identify and make considerations about known design limitations of Alpaca and Pico Pi

> <font color='grey'>⏳ Estimated time: 5 + 60 + 30  min</font>

### Range and Resolution

What is the available range and resolution of: 

1. Pico Pi's analog-to-digital converters (ADC)?
2. Alpaca's digital-to-analog converters (DAC)?

### Device timer

Recall the smallest value of `DELAY_MS` used in your previous experiments with Pico.

1. What is then the highest sampling rate achievable with Pico?
2. 🏆 **Optional:** Do you think it is reliable and consistent?<br>
    > In one of the experiments in the notebook 17B, you will have a chance to explore some timing peculiarities and timing inconsistencies of Pico Pi.

### Opamps

1. What are the highest and the lowest outputs of the OPAMPs that you **measured** in the previous assignments? <br> *Keep this in mind throughout the project and also when answering the questions in A2.*
2. Are you expecting the $U_{\text{offset}}$ and $I_{\text{bias}}$ of the OPAMPs to affect your measurement significantly?

## A2: Predict signals

We would to encourage you to choose the *Advanced Design* for this project. However, if you decide to go with the *Basic Design*, you may consider the tasks in the *Advanced* track as optional.

- **A2.1: Required**
- **A2.2: Required only for the Advanced design - Recommended!**

### A2.1 Design B (Basic) - Linear Potentiostat Model

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/DesignBModel1+ADC.png"><br>
    <em>Linear Potentiostat Model - Basic Design</em>
</div><br>

1. Recall the answer from *Anticipate1: Resolution* and compute the smallest $U_{\text{cell}}$ step when *DAC Assistant* is used to control it.

2. Derive the formula for $U_a$ as a function of $U_{DAC A}$

3. Compute the range of $U_{DAC A}$ required to generate $U_{DAC Assistant} = (-3V ,3V)$ 

4. Compute the range of $I_{cell}$ generated by $U_{DAC Assistant} = (-3V ,3V)$ for:

   1. $R_1 = R_2 = 1k\Omega$
   2. $R_1 = R_2 = 10k\Omega$

5. Compute the ranges of $U_a$ generated by $U_{DAC Assistant} = (-3V ,3V)$ for:

    1. $R_1 = R_2 = 1k\Omega$ and $R_f = 1k\Omega$
    2. $R_1 = R_2 = 1k\Omega$ and $R_f = 3.3k\Omega$
    3. $R_1 = R_2 = 10k\Omega$ and $R_f = 10k\Omega$
    4. $R_1 = R_2 = 10k\Omega$ and $R_f = 33k\Omega$

5. Which attenuation and the jumper setting (A,B,C,D,E) on Alpaca's `ATTENUATOR` on `J42` of the `AMP0` would you choose to make the best use of the available range of `ADC0=(0,3.3V)` when capturing the positive signals via `J40:SIGNAL+`? Answer the question for each of the cases above
   <details>
   <summary><font size=4>💡</font> <b>Hint</b></summary>

   > - What is the highest value of `Ua` possible?
   > - What is the highest value of `Ua` that can be measured at each attenuation factor?
   > - Use the relevant table from the Alpaca Manual in section  *2.10 The Amplifier* to match the attenuation with the jumper setting.
   </details>
   
   

6. You will use `AMP1` to measure the negative part of the signal via `J60:SIGNAL-`. In the default setting - recommended - the inverting amplifier `AMP1:SIGNAL-` has a resistor `R69=100k` on the feedback line. So, in order to use it as an attenuator, you can simply insert a resistor between the output `Ua` and `AMP1:SIGNAL-`.
   - Calculate the resistor $R_{preamp}$ that you need to add to your circuit on the breadboard between `Ua` and `J60:SIGNAL-` to match the attenuation selected for the positive inputs via `ADC0` in the previous question.

 <details><summary><font size=4>💡</font> <b>Using AMP1 as an inverting attenuator</b></summary> 

>
> If you have a look into the Alpaca's circuit diagram in the Alpaca Manual, the last page, and find the circuit of the amplifier associated with `ADC1` (*Hint: Look for jumpers and resistors in series *60*, i.e. `J62` or `R69` to find `AMP1`*), you will notice that the `J62:ATTENUATOR` only works for signals measured via `J60:SIGNAL+`.
>
> Reading and understanding Alpaca's circuit diagram may seem daunting and complicated at first, but it turns out that in order to attenuate a negative input and match the attenuation on the positive part of the signal, you will simply have to add an extra resistor before the inverting input of the OPAMP.  
> 
> The default setting for `J60:SIGNAL-` is `GAIN=-1` and it is realised with the following combination:
> 1. `Ua` to `J60:SIGNAL- = J60:3` 
> 2. `J60:SIGNAL+ = J60:2` to `J60:GND = J60:1`
> 3. `J60:DC`
> 5. `J62:1-4 = J62:B = 0 Ohm`
> 6. `J63:1-4 = J63:B = 0 Ohm`
> 7. `J65:1-4 = J65:B = 0 Ohm`
> 8. No jumpers on `J64` and `J17`
</details>

### A2.2 Design A (Advanced) - Linear Potentiostat Model

<div style="text-align: center">
<img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/DesignAModel1+ADC.png"><br>
<em>Linear Potentiostat Model - Advanced Design</em>
</div><br>


1. Recall the answer from *Anticipate1: Resolution* and compute the smallest $U_{\text{cell}}$ step when `DAC A` and `DAC B` are used to control it.

2. Derive the formula for `Ua` as a function of `DAC A` and `DAC B`, and calculate the maximum range of $U_{cell}$ achievable with this configuration.

3. There are at least three procedures to control `Ucell` and operate the potentiostat:

   >1. Use `DAC A` to fix the potential of the `Ref` and use `DAC B` to control the `Ucell`
   >2. Use `DAC B` to fix the potential of the `WE` and use `DAC A` to control the `Ucell`
   >3. Modulate `DAC A` and `DAC B` simultaneously to control the `Ucell`
   
   Each procedure has its pros and cons when used to perform a voltage sweep across the Measurement Cell. <br><br>

   🏆🏆🏆 **Optional:** List pros and cons for each control procedure mentioned above.<br><br>

    Use procedure 2 in this and in the following exercises. Take the range $-0.5 V$ to $1.5 V$, so $U_{cell} = (-0.5V,1.5V)$ as an example. You may assume that $R_1 = R_2 = 1k\Omega$ and $R_f = 3.3k\Omega$. Remember that you can only measure positive voltages at `Ua`. For simplicity or to optimise for resolution and signal-to-noise ratio, we will use attenuations `1:1` and `1:3` via `ADC0:SIGNAL+`, depending on the priority.
   - Derive the formula to find the range of $I_{cell}$,
   - Compute the range for both attenuation factors when `DAC B = 1.5V`.<BR>
   - 🏆**Optional:** Derive the general formula for the effective resolution of $I_{cell}$ and compute it for both attenuations factors.
   

    <details><summary><font size=4>💡</font> <b>Hints</b></summary>

    > - What is the highest value of `Ua` possible?
    > - What is the highest value of `Ua` that can be measured at each attenuation factor?
    > - Use the relevant table from the Alpaca Manual in section  *2.10 The Amplifier* to match the attenuation with the jumper setting for `J42`.</details>